# Acessando dados do Nubank usando Cognos Jupyter Notebook

In [ ]:
#instalação da biblioteca:
!pip install pynubank --user
from pynubank import Nubank
import getpass
import pandas as pd
import numpy as np

# Diretório do COGNOS ANALYTICS:
cm_folder = ".public_folders/pyNUBANK/"

nu = Nubank()
uuid, qr_code = nu.get_qr_code()

## 1) Autenticação

### 1.1 CPF:

In [ ]:
# Utilize o CPF sem pontos ou traços
CPF  = input('Seu CPF usado no cadastro do site do Nubank:')

### 1.2 SENHA: 

In [ ]:
# Utilize o CPF sem pontos ou traços
SENHA = getpass.getpass("Digite a senha do site do Nubank:")

### 1.3 Authenticação de 2 fatores

In [ ]:
from pynubank import Nubank
nu = Nubank()
uuid, qr_code = nu.get_qr_code()
qr_code.print_ascii(invert=True)
input('Após escanear o QRCode pressione enter para continuar')
nu.authenticate_with_qr_code(CPF, SENHA, uuid)

## 2) Detalhamento do Cartão de Crédito

In [ ]:
# Lista de dicionários contendo todas as transações de seu cartão de crédito
card_statements = nu.get_card_statements()

df_credito = pd.DataFrame(card_statements)
df_credito['time'] = pd.to_datetime(df_credito['time'])
## Concatenação com o df_credito.details formatado
df_credito_full = pd.concat([df_credito,pd.io.json.json_normalize(df_credito.details)], axis=1)
df_credito_full['amount'] = (df_credito_full['amount'] / 100).map('{:.2f}'.format)
df_credito_full['amount_without_iof'] = (df_credito_full['amount_without_iof'] / 100).map('{:.2f}'.format)

# Enviar dados para o COGNOS ANALYTICS:
file = "cartao_credito"
CADataConnector.write_data(df_credito_full, path=cm_folder+file, mode="w")

## 3) Faturas do cartão de crédito

In [ ]:
bills = nu.get_bills()
df_faturas = pd.DataFrame(bills)
df_faturas_2 = pd.io.json.json_normalize((df_faturas.summary))
df_faturas_full = pd.concat([df_faturas['state'],df_faturas_2], axis=1)

df_faturas_full['open_date'] = pd.to_datetime(df_faturas_full['open_date'])
df_faturas_full['total_balance'] = (df_faturas_full['total_balance'] / 100).map('{:.2f}'.format)
df_faturas_full['total_cumulative'] = (df_faturas_full['total_cumulative'] / 100).map('{:.2f}'.format)
df_faturas_full['past_balance'] = (df_faturas_full['past_balance'] / 100).map('{:.2f}'.format)

# Enviar dados para o COGNOS ANALYTICS:
file = "cartao_faturas"
CADataConnector.write_data(df_faturas_full, path=cm_folder+file, mode="w")

## 4) Detalhamento da NuConta

In [ ]:
transactions = nu.get_account_statements()
df_conta = pd.DataFrame(transactions)
df_conta['postDate'] = pd.to_datetime(df_conta['postDate'])

conditions = [
    (df_conta.title == 'Compra no débito'),
    (df_conta.title == 'Depósito recebido'),
    (df_conta.title == 'Pagamento da fatura'),
    (df_conta.title == 'Transferência enviada'),
    (df_conta.title == 'Transferência recebida')]
values = [df_conta.amount * -1, df_conta.amount, df_conta.amount * -1, df_conta.amount * -1, df_conta.amount]
df_conta['balanco'] = np.select(conditions, values)


# Enviar dados para o COGNOS ANALYTICS:
file = "conta"
CADataConnector.write_data(df_conta, path=cm_folder+file, mode="w")